In [0]:
import pandas as pd

In [0]:
# Load tables from the catalog
crossborder = spark.table("curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw.crossborder_flows__fixed")
generation = spark.table("curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw.generation")

capacity = spark.table("curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw.installed_capacity")

load_actual = spark.table("curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw.load_actual")
load_forecast = spark.table("curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw.load_forecast")

solar_forecast = spark.table("curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw.solar_forecast")
wind_forecast = spark.table("curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw.wind_forecast")

In [0]:
print("load_actual:", load_actual.columns)
print("load_forecast:", load_forecast.columns)
print("generation:", generation.columns)
print("crossborder:", crossborder.columns)
print("solar_forecast:", solar_forecast.columns)
print("wind_forecast:", wind_forecast.columns)
print("capacity:", capacity.columns)

In [0]:
df = load_actual
df = df.join(load_forecast, on=["index", "country"], how="outer")
df = df.join(solar_forecast, on=["index", "country"], how="outer")
df = df.join(wind_forecast, on=["index", "country"], how="outer")
df = df.join(generation, on=["index", "country"], how="outer")
df = df.join(capacity, on=["index", "country"], how="outer")

In [0]:
display(df.limit(5))

In [0]:

# Rename flow column
flows = crossborder.withColumnRenamed("_0", "flow_mw")

# Compute imports
imports = (flows
           .groupBy("to_country")
           .sum("flow_mw")
           .withColumnRenamed("to_country", "country")
           .withColumnRenamed("sum(flow_mw)", "import_mw"))

# Compute exports
exports = (flows
           .groupBy("from_country")
           .sum("flow_mw")
           .withColumnRenamed("from_country", "country")
           .withColumnRenamed("sum(flow_mw)", "export_mw"))

# Join both into your main df
df = df.join(imports, on="country", how="outer")
df = df.join(exports, on="country", how="outer")

In [0]:
df_pandas = df.toPandas()